In [1]:
import requests

In [2]:
import geopandas as gpd
import numpy as np
import pandas as pd

## Open State data

In [3]:
file_path = "data/cb_2024_us_county_500k/cb_2024_us_county_500k.shp"
counties_gdf = gpd.read_file(file_path)

## Get Columm Data

In [4]:
r = requests.get("https://api.census.gov/data/2023/acs/acs5/groups/B25103.json")
columns_obj = r.json()

In [5]:
columns = []
rename_vars = {}
variables = columns_obj["variables"]
for name, variable in list(variables.items()):
    v_split = variable["label"].split("!!")
    if len(v_split) < 3:
        continue

    if v_split[0] == "Estimate":
        label = v_split[-1]
        rename_vars[name] = label

    if (name.endswith("E") or name.endswith("M")) and v_split[
        -1
    ] == "Median real estate taxes paid for units without a mortgage":
        columns.append(name)

In [6]:
columns.append("GEO_ID")
columns_formatted = ",".join(columns)

In [7]:
response = requests.get(
    f"https://api.census.gov/data/2023/acs/acs5?get={columns_formatted}&for=county:*"
)

In [8]:
data = response.json()
columns = data[0]
rows = data[1:]
df = pd.DataFrame(rows, columns=columns)

In [16]:
col_vars = {
    k: v["label"].split("!!")[-1]
    for k, v in variables.items()
    if k.startswith("B25103")
}
col_vars["GEO_ID"] = "GEOIDFQ"

In [17]:
estimate_cols = [col for col in df.columns if col.endswith("E")]

In [18]:
formtted_df = df[["GEO_ID", *estimate_cols]]
formtted_df[estimate_cols] = formtted_df[estimate_cols].astype(int)

/tmp/ipykernel_11973/4148564808.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  formtted_df[estimate_cols] = formtted_df[estimate_cols].astype(int)


In [19]:
formtted_df = formtted_df.rename(columns=col_vars)

In [20]:
formtted_df = formtted_df.replace(-666666666, np.nan)

## Merge Data

In [22]:
mortgage_gdf = counties_gdf.merge(formtted_df, on="GEOIDFQ", how="inner")

In [23]:
mortgage_gdf = mortgage_gdf.to_crs(9311)
mortgage_gdf.to_file(
    "data/Median_Real_Estate_Taxes_Paid_Without_Mortgage_Per_County.gpkg"
)